In [1]:
!pip install colorama
!pip install -U tensorly
!pip install -U tensorly-torch

     |████████████████████████████████| 198 kB 4.1 MB/s 
     |████████████████████████████████| 154 kB 40.0 MB/s 
     |████████████████████████████████| 52 kB 885 kB/s 


In [6]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab-Notebooks/tensor_adapt_165
# %cd /content/drive/MyDrive/courses/CS165/tensor_adapt


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab-Notebooks/tensor_adapt_165


### Debug


In [3]:
%cd /content/drive/MyDrive/Colab-Notebooks/tensor_adapt_165

from __future__ import print_function
import argparse

import torch
import torch.nn as nn
import torch.optim as optim

from utils.misc import *
from utils.test_helpers_TTT import *
from utils.prepare_dataset import *
from utils.rotation import rotate_batch
from utils.load_weights_ckp import *

parser = argparse.ArgumentParser()
parser.add_argument('--dataset', default='cifar10')
parser.add_argument('--dataroot', default='datasets/')
parser.add_argument('--shared', default='layer2')
########################################################################
parser.add_argument('--depth', default=26, type=int)
parser.add_argument('--width', default=1, type=int)
parser.add_argument('--batch_size', default=128, type=int)
parser.add_argument('--group_norm', default=8, type=int)
parser.add_argument('--tensor', default=True)
########################################################################
parser.add_argument('--lr', default=0.1, type=float)
parser.add_argument('--nepoch', default=150, type=int)
parser.add_argument('--milestone_1', default=75, type=int)
parser.add_argument('--milestone_2', default=125, type=int)
parser.add_argument('--rotation_type', default='expand')
########################################################################
parser.add_argument('--outf', default='.')

args = parser.parse_args(args=[])

#cudnn.benchmark = True
net, ext, head, ssh = build_model_modules(args)
_, teloader = prepare_test_data(args)
_, trloader = prepare_train_data(args)

parameters = list(net.parameters()) + list(head.parameters())
optimizer = optim.SGD(parameters, lr=args.lr, momentum=0.9, weight_decay=5e-4)

print("net's state_dict:")
for param_tensor in net.state_dict():
    print(param_tensor, "\t", net.state_dict()[param_tensor].size())

/content/drive/MyDrive/Colab-Notebooks/tensor_adapt_165


ModuleNotFoundError: ignored

In [ ]:
%cd /content/drive/MyDrive/Colab-Notebooks/tensor_adapt_165

from __future__ import print_function
import argparse

import torch
import torch.nn as nn
import torch.optim as optim

from utils.misc import *
from utils.test_helpers_TTT import *
from utils.prepare_dataset import *
from utils.rotation import rotate_batch
from utils.load_weights_ckp import *

parser = argparse.ArgumentParser()
parser.add_argument('--dataset', default='cifar10')
parser.add_argument('--dataroot', default='datasets/')
parser.add_argument('--shared', default='layer2')
########################################################################
parser.add_argument('--depth', default=26, type=int)
parser.add_argument('--width', default=1, type=int)
parser.add_argument('--batch_size', default=128, type=int)
parser.add_argument('--group_norm', default=8, type=int)
parser.add_argument('--tensor', default=True)
########################################################################
parser.add_argument('--lr', default=0.1, type=float)
parser.add_argument('--nepoch', default=150, type=int)
parser.add_argument('--milestone_1', default=75, type=int)
parser.add_argument('--milestone_2', default=125, type=int)
parser.add_argument('--rotation_type', default='rand')
########################################################################
parser.add_argument('--outf', default='.')

args = parser.parse_args(args=[])

net, ext, head, ssh = build_model_modules(args)
_, teloader = prepare_test_data(args)
_, trloader = prepare_train_data(args)

In [18]:
_, teloader = prepare_test_data(args)
_, trloader = prepare_train_data(args)

# load from checkpoint to resume training
checkpoint_dir = 'results/cifar10_tensor_layer2_gn_expand/ckpt.pth'
ckpt = torch.load(checkpoint_dir)
net.load_state_dict(ckpt['net'], strict=False)
head.load_state_dict(ckpt['head'], strict=False)
print('resume training from checkpoint ' + str(checkpoint_dir))

# create (50k x rank) tensors, store on cpu
u_train_net = []
u_test_net = []
u_train_head = []
u_test_head = []

for param_name, param in net.named_parameters():
    if 'weight.weights' in param_name:
        u_train_net.append(torch.ones((len(trloader.dataset), len(param)), pin_memory=True))
        u_test_net.append(torch.ones((len(teloader.dataset), len(param)), pin_memory=True))

for param_name, param in head.named_parameters():
    if 'weight.weights' in param_name:
        u_train_head.append(torch.ones((len(trloader.dataset), len(param)), pin_memory=True))
        u_test_head.append(torch.ones((len(teloader.dataset), len(param)), pin_memory=True))

parameters = list(net.parameters()) + list(head.parameters())
optimizer = optim.SGD(parameters, lr=args.lr, momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.MultiStepLR(
    optimizer, [args.milestone_1, args.milestone_2], gamma=0.1, last_epoch=-1)
criterion = nn.CrossEntropyLoss().cuda()

print("net's state_dict:")
for param_tensor in net.state_dict():
    print(param_tensor, "\t", net.state_dict()[param_tensor].size())

print("ssh's state_dict:")
for param_tensor in ssh.state_dict():
    print(param_tensor, "\t", ssh.state_dict()[param_tensor].size())

all_err_cls = []
all_err_ssh = []
all_loss = []
all_loss_ssh = []
print('Running...')
print('Error (%)\t\ttest\t\tself-supervised\t\tloss_last_batch\t\tloss_ssh_last_batch')
for epoch in range(1, args.nepoch + 1):
    net.train()
    ssh.train()

    for batch_idx, (inputs, labels, data_idx) in enumerate(trloader):
      if batch_idx==
        optimizer.zero_grad()
        inputs_cls, labels_cls = inputs.cuda(), labels.cuda()

        u_train_net_data = [u_train_net[ll][data_idx, :].cuda() for ll in range(len(u_train_net))]
        u_train_head_data = [u_train_head[ll][data_idx, :].cuda() for ll in range(len(u_train_head))]
        load_trainable_weights(net, u_train_net_data)
        load_trainable_weights(head, u_train_head_data)

        outputs_cls = net(inputs_cls, adapt=True)
        loss = criterion(outputs_cls, labels_cls)

        if args.shared is not None:
            inputs_ssh, labels_ssh = rotate_batch(inputs, args.rotation_type)
            inputs_ssh, labels_ssh = inputs_ssh.cuda(), labels_ssh.cuda()
            outputs_ssh = ssh(inputs_ssh, adapt=True)
            loss_ssh = criterion(outputs_ssh, labels_ssh)
            loss += loss_ssh

        loss.backward()
        optimizer.step()

        u_train_net_data = save_trainable_weights(net)
        u_train_head_data = save_trainable_weights(head)

        for ll in range(len(u_train_net_data)):
            u_train_net[ll][data_idx, :] = u_train_net_data[ll].clone().detach().cpu()
        for ll in range(len(u_train_head_data)):
            u_train_head[ll][data_idx, :] = u_train_head_data[ll].clone().detach().cpu()

    err_cls = test(teloader, net, adapt=True)[0]
    err_ssh = 0 if args.shared is None else test(teloader, ssh, sslabel='expand', adapt=True)[0]
    all_err_cls.append(err_cls)
    all_err_ssh.append(err_ssh)
    all_loss.append(loss)
    all_loss_ssh.append(loss_ssh)
    scheduler.step()

    print(('Epoch %d/%d:' % (epoch, args.nepoch)).ljust(24) +
          '%.2f\t\t%.2f\t\t%.2f\t\t%.2f' % (err_cls * 100, err_ssh * 100, loss, loss_ssh))
    torch.save((all_err_cls, all_err_ssh), args.outf + '/loss.pth')
    torch.save((all_loss, all_loss_ssh), args.outf + '/train_loss_last_batch.pth')
    plot_epochs(all_err_cls, all_err_ssh, args.outf + '/loss.pdf')

    if epoch % 5 == 0:
        checkpoint_dir = args.outf + '/checkpoint' + str(epoch) + '.pt'
        print(checkpoint_dir)
        checkpoint = {
            'epoch': epoch + 1,
            'net': net.state_dict(),
            'head': head.state_dict(),
            'optimizer': optimizer.state_dict()
        }
        save_ckp(checkpoint, checkpoint_dir)

state = {'err_cls': err_cls, 'err_ssh': err_ssh,
         'net': net.state_dict(), 'head': head.state_dict(),
         'optimizer': optimizer.state_dict()}
torch.save(state, args.outf + '/ckpt.pth')

resume training from checkpoint results/cifar10_tensor_layer2_gn_expand/ckpt.pth
net's state_dict:
conv1.adaptive_weights_preconv_trainable 	 torch.Size([128, 19])
conv1.conv1.weight.weights 	 torch.Size([19])
conv1.conv1.weight.factors.factor_0 	 torch.Size([16, 19])
conv1.conv1.weight.factors.factor_1 	 torch.Size([3, 19])
conv1.conv1.weight.factors.factor_2 	 torch.Size([3, 19])
conv1.conv1.weight.factors.factor_3 	 torch.Size([3, 19])
layer1.0.adaptive_weights_conv1_trainable 	 torch.Size([128, 67])
layer1.0.adaptive_weights_conv2_trainable 	 torch.Size([128, 67])
layer1.0.bn1.weight 	 torch.Size([16])
layer1.0.bn1.bias 	 torch.Size([16])
layer1.0.conv1.weight.weights 	 torch.Size([67])
layer1.0.conv1.weight.factors.factor_0 	 torch.Size([16, 67])
layer1.0.conv1.weight.factors.factor_1 	 torch.Size([16, 67])
layer1.0.conv1.weight.factors.factor_2 	 torch.Size([3, 67])
layer1.0.conv1.weight.factors.factor_3 	 torch.Size([3, 67])
layer1.0.bn2.weight 	 torch.Size([16])
layer1.0.bn2.bi

RuntimeError: ignored

In [20]:
batch_idx

390

## Training

In [ ]:
%%shell
export PYTHONPATH=$PYTHONPATH:$(pwd)

CUDA_VISIBLE_DEVICES=0 python /content/drive/MyDrive/courses/CS165/tensor_adapt/main.py --tensor --shared layer2 --rotation_type expand \
			--group_norm 8 \
			--nepoch 150 --milestone_1 75 --milestone_2 125 \
			--batch_size 64 \
			--outf results/cifar10_tensor_layer2_gn_expand

/usr/local/lib/python3.7/dist-packages/tltorch/factorized_tensors/core.py:145: UserWarning: Creating a subclass of FactorizedTensor TensorizedTensor with no name.
  warnings.warn(f'Creating a subclass of FactorizedTensor {cls.__name__} with no name.')
Building model...
Test on the original test set
Files already downloaded and verified
Preparing data...
Files already downloaded and verified
Running...
Error (%)		test		self-supervised
Epoch 1/150:            90.00		75.00
Epoch 2/150:            90.00		75.00
Epoch 3/150:            90.00		75.00
Traceback (most recent call last):
  File "/content/drive/MyDrive/courses/CS165/tensor_adapt/main.py", line 61, in <module>
    outputs_cls = net(inputs_cls)
  File "/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py", line 1102, in _call_impl
    return forward_call(*input, **kwargs)
  File "/content/drive/MyDrive/courses/CS165/tensor_adapt/models/ResNet_tensor_adapt_init.py", line 132, in forward
    x = self.layer3(x, adapt=adapt

CalledProcessError: ignored

In [ ]:
%%shell
export PYTHONPATH=$PYTHONPATH:$(pwd)

CUDA_VISIBLE_DEVICES=0 python /content/drive/MyDrive/Colab-Notebooks/tensor_adapt/main.py --tensor --shared layer2 --rotation_type expand \
			--group_norm 8 \
			--nepoch 150 --milestone_1 75 --milestone_2 125 \
			--batch_size 64 \
			--outf results/cifar10_tensor_layer2_gn_expand

/usr/local/lib/python3.7/dist-packages/tltorch/factorized_tensors/core.py:145: UserWarning: Creating a subclass of FactorizedTensor TensorizedTensor with no name.
  warnings.warn(f'Creating a subclass of FactorizedTensor {cls.__name__} with no name.')
Building model...
/usr/local/lib/python3.7/dist-packages/tensorly/backend/core.py:1106: UserWarning: In partial_svd: converting to NumPy. Check SVD_FUNS for available alternatives if you want to avoid this.
  warnings.warn('In partial_svd: converting to NumPy.'
Test on the original test set
Files already downloaded and verified
Preparing data...
Files already downloaded and verified
Model's state_dict:
conv1.adaptive_weights_preconv 	 torch.Size([1, 17])
conv1.conv1.weight.weights 	 torch.Size([17])
conv1.conv1.weight.factors.factor_0 	 torch.Size([16, 17])
conv1.conv1.weight.factors.factor_1 	 torch.Size([3, 17])
conv1.conv1.weight.factors.factor_2 	 torch.Size([3, 17])
conv1.conv1.weight.factors.factor_3 	 torch.Size([3, 17])
layer1.0.a